# 07a_slope_to_edges (Guadalajara only)

## Import libraries

In [1]:
first_folder_path = '../../../'

In [2]:
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
from geopandas.tools import overlay
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import osmnx as ox
import rasterio
from rasterio.plot import show

import os
import sys
module_path = os.path.abspath(os.path.join(first_folder_path))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
else:
    import aup

## Notebook config

In [3]:
# ----- ----- ----- Projection to be used when needed ----- ----- -----
projected_crs = "EPSG:32613"

# Location of unzipped MDE data by state obtained from https://www.inegi.org.mx/app/geo2/elevacionesmex/
grl_path = first_folder_path + 'data/external/MDE/'

## Load data

### __Load data__ - Nodes and edges

In [ ]:
# Read file
nodes = gpd.read_file(first_folder_path + "data/processed/volvo/project_network_initial_guadalajara/guadalajara_nodes_proj_net_rebuilt.gpkg")
# Filter for data of interest
nodes = nodes[['osmid','x','y','street_count','geometry']]
# Turn to EPSG:4326
nodes = nodes.to_crs("EPSG:4326")
# Add x and y data
nodes['x'] = nodes['geometry'].x
nodes['y'] = nodes['geometry'].y

# Show
print(nodes.crs)
print(nodes.shape)
nodes.head(2)

In [ ]:
# Read file
edges = gpd.read_file(first_folder_path + "data/processed/volvo/project_network_initial_guadalajara/guadalajara_edges_proj_net_rebuilt.gpkg")
# Filter for data of interest
edges = edges[['u','v','key','geometry']]
# Add length data
if edges.crs != projected_crs:
    edges = edges.to_crs(projected_crs)
    print(f"Changed crs to {projected_crs}.")
edges['length'] = edges.length
# Turn to EPSG:4326
edges = edges.to_crs("EPSG:4326")

# Show
print(edges.crs)
print(edges.shape)
edges.head(2)

### __Load data__ - Load and show DEM raster

In [ ]:
mde_path = first_folder_path + "data/processed/volvo/slope_to_edges_gdl/guadalajaramunsbuffer_r15m_4326.tif"

fig,ax = plt.subplots(1,1)

# Nodes
print(f"Nodes crs: {nodes.crs}.")
nodes.plot(ax=ax,zorder=1,color='black',markersize=1)
print(f"Edges crs: {nodes.crs}.")
edges.plot(ax=ax,zorder=0,color='lightgrey',linewidth=1)

with rasterio.open(mde_path) as mde:
    print(f"Raster crs: {mde.crs}.")
    show(mde,cmap='Reds',ax=ax)

## Create G from nodes and edges

In [ ]:
# Prepare nodes and edges
if 'osmid' in nodes.columns:
    print(f"Nodes before dropping dups: {len(nodes)}.")
    nodes.drop_duplicates(subset='osmid',inplace=True)
    print(f"Nodes after dropping dups: {len(nodes)}.")
    nodes.set_index('osmid',inplace=True)

if 'u' in edges.columns:
    print(f"Edges before dropping dups: {len(edges)}.")
    edges.drop_duplicates(subset=['u','v','key'],inplace=True)
    print(f"Edges after dropping dups: {len(edges)}.")
    edges.set_index(['u','v','key'],inplace=True)

# Create graph
G = ox.convert.graph_from_gdfs(nodes, edges)
G

In [ ]:
nodes.head(2)

In [ ]:
edges.head(2)

## Add slope to nodes and edges

In [ ]:
mde_path

In [ ]:
#elevations to nodes
G_elev_mde = ox.elevation.add_node_elevations_raster(G, mde_path,band=1)
assert not np.isnan(np.array(G_elev_mde.nodes(data="elevation"))[:, 1]).any()

In [ ]:
nodes_df = ox.graph_to_gdfs(G_elev_mde, nodes=True, edges=False)

print(f"Nodes: {len(nodes_df)}.")
print(f"Nodes without elevation data: {nodes_df[['elevation']].isna().sum()[0]}.")

In [ ]:
# Sloped length to edges
G_elev_mde = ox.distance.add_edge_lengths(G_elev_mde)
# Slope to edges
G_elev_mde = ox.elevation.add_edge_grades(G_elev_mde, add_absolute=True, precision=3)
# G to nodes and edges
nodes_elev_mde, edges_elev_mde = ox.graph_to_gdfs(G_elev_mde, nodes=True, edges=True)

# LOG CODE - Print progress of script so far
mean_elev = round(nodes_elev_mde.elevation.mean(),2)
mean_slope = round(edges_elev_mde.grade_abs.mean(),2)
print(f"--- Assigned a mean elevation of {mean_elev} to nodes")
print(f"    and mean slope of {mean_slope} to edges.")

# reset index for upload
nodes_elev_mde.reset_index(inplace=True)
edges_elev_mde.reset_index(inplace=True)

#set street_count as float
nodes_elev_mde["street_count"] = nodes_elev_mde["street_count"].astype(float)

# Show
print(nodes_elev_mde.shape)
nodes_elev_mde.head(2)

In [ ]:
# Show
print(edges_elev_mde.shape)
edges_elev_mde.head(2)